In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# 시드설정
random.seed(42)
torch.manual_seed(42)
if device == 'cuda':
    torch.cuda.manual_seed_all(42)

cpu


### Get data

In [2]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

9920512it [00:02, 4867085.45it/s]                             


Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 63052.59it/s]            


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:01, 1603951.52it/s]                             


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 28943.44it/s]            

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!



/Users/mglee/miniforge3/envs/mglee/lib/python3.9/site-packages/torchvision/datasets/mnist.py:335: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /tmp/pip-req-build-wdh4qigd/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed).view(length, num_rows, num_cols)


In [13]:
# Set parameter

learning_rate = 0.001
training_epochs = 10
batch_size = 32

### Create Data Loader

In [5]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

### Create layers

In [7]:
linear1 = torch.nn.Linear(784, 128, bias=True)
linear2 = torch.nn.Linear(128, 4, bias=True)
linear3 = torch.nn.Linear(4, 10, bias=True)

sigmoid = torch.nn.Sigmoid() #activation function으로 ReLU 설정

In [8]:
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

Parameter containing:
tensor([[-0.1432,  0.0503,  0.3856, -0.6233],
        [ 0.5856,  0.0341, -0.3474,  0.2400],
        [ 0.6105, -0.5592, -0.3224,  0.3533],
        [-0.2542,  0.5842,  0.0181, -0.2762],
        [-0.2787, -0.1097, -0.1010,  0.3929],
        [-0.5447,  0.3612,  0.6122,  0.5719],
        [-0.5944, -0.4872,  0.4359,  0.6332],
        [ 0.6087,  0.4257,  0.4043, -0.3691],
        [-0.5996, -0.2346,  0.2869, -0.0458],
        [ 0.1125,  0.6066,  0.2235,  0.3242]], requires_grad=True)

In [9]:
# Sequential model

model = torch.nn.Sequential(linear1, sigmoid,
                            linear2, sigmoid,
                            linear3
                            )

In [11]:
#Define cost F and optimizer

error = torch.nn.CrossEntropyLoss()    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
total_batch = len(data_loader)
model.train()    # set the model to train mode (dropout=True)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.view(-1, 28 * 28)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = error(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.786825299
Epoch: 0002 cost = 0.514728665
Epoch: 0003 cost = 0.384559661
Epoch: 0004 cost = 0.300640583
Epoch: 0005 cost = 0.239592478
Epoch: 0006 cost = 0.197646305
Epoch: 0007 cost = 0.165636078
Epoch: 0008 cost = 0.136867061
Epoch: 0009 cost = 0.112623155
Epoch: 0010 cost = 0.092561990
Learning finished


In [ ]:
with torch.no_grad():
    model.eval()    # set the model to evaluation mode (dropout=False)

    # Test the model using test sets
    X_test = mnist_test.test_data.view(-1, 28 * 28).float()
    Y_test = mnist_test.test_labels

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())
    
    # 임의의 자료 가져와 일치여부 확인
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())